In [143]:
import pandas as pd

In [144]:
df = pd.read_csv('/Users/estorrs/Downloads/13059_2014_3364_MOESM1_ESM/supplementary_table_2-Table 1.csv')

In [145]:
df = df[['CHR', 'POS', 'VariationName', 'VT', 'VLENGTH', 'Ancestralallele', 'Consequencespecificallele']]

In [146]:
df.head(10)

,CHR,POS,VariationName,VT,VLENGTH,Ancestralallele,Consequencespecificallele
0,13,106835388,rs9519977,SNP,1,A,na
1,1,116935068,rs10924081,SNP,1,G,A/G
2,3,146978905,rs4059271,INDEL,3,na,T/G
3,8,140933740,rs55726039,SNP,1,T,T/C
4,3,5775231,rs1811510,SNP,1,T,na
5,10,68591682,rs10762111,SNP,1,A,T/A
6,22,50704357,rs11704249,SNP,1,C,C/T
7,2,235946882,na,INDEL,3,na,na
8,1,91200757,rs76306981,INDEL,2,na,na
9,10,111924518,rs11814770,SNP,1,T,na


In [147]:
def get_alternate(reference, consequence):
    pieces = consequence.split('/')
    if pieces[0] == reference:
        return pieces[1]
    return pieces[0]

def populate_alternate(df):
    filtered = pd.DataFrame()
    for i, row in df.iterrows():
        # just do snps for now
        if row['VLENGTH'] == 1:
            if row['Ancestralallele'] != 'na' and row['Consequencespecificallele'] != 'na':
                row['Alternate'] = get_alternate(row['Ancestralallele'], row['Consequencespecificallele'])
                filtered = filtered.append(row)
    return filtered
                
        

In [148]:
df = df.astype({'POS': str, 'CHR': str})

In [149]:
df = populate_alternate(df)

In [150]:
position_df = df[['CHR', 'POS']]
alleles_df = df[['CHR', 'POS', 'Ancestralallele', 'Alternate']]

In [151]:
alleles_df[df['CHR']=='22']

,CHR,POS,Ancestralallele,Alternate
6,22,50704357,C,T
74,22,48228760,C,T
215,22,45549032,T,C
280,22,46771921,C,T
389,22,25914727,G,C
497,22,22547460,T,C
830,22,35534933,G,C
863,22,46500164,C,G
956,22,45279529,A,G
1005,22,29854959,T,C


In [152]:
position_df.to_csv(open('/Users/estorrs/Documents/ding/ancestry/positions.tsv', 'w'), sep='\t', header=None, index=None)
alleles_df.to_csv(open('/Users/estorrs/Documents/ding/ancestry/alleles.tsv', 'w'), sep='\t', header=None, index=None)

In [169]:
def get_vcf_df(vcf_fn):
    f = open(vcf_fn)
    header = ''
    vcf_header = ''
    for line in f:
        vcf_header += line
        if 'CHROM\tPOS\tID' in line:
            header = line
            break

    df = pd.read_csv(f, delimiter='\t', header=None)
    headers = header.split('\t')
    
    # remove # if necessary
    if headers[0][0] == '#':
        headers[0] = headers[0][1:]

    df.columns = headers
    
    return df, vcf_header

In [170]:
allele_df = pd.read_csv(open('/Users/estorrs/Documents/ding/ancestry/ancestry-tool/app/tests/data/alleles.tsv'),
                        delimiter='\t', header=None)
vcf_df, vcf_header = get_vcf_df('/Users/estorrs/Documents/ding/ancestry/ancestry-tool/app/tests/data/example.vcf')

In [171]:
vcf_df.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG00096,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,22,16050075,rs587697622,A,G,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=8012;EA...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1,22,16050115,rs587755077,G,A,100,PASS,AC=32;AF=0.00638978;AN=5008;NS=2504;DP=11468;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,22,16050213,rs587654921,C,T,100,PASS,AC=38;AF=0.00758786;AN=5008;NS=2504;DP=15092;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
3,22,16050319,rs587712275,C,T,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=22609;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
4,22,16050527,rs587769434,C,A,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=23591;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [172]:
# attach headers to allele dataframe
allele_df.columns = ['CHROM', 'POS', 'REF', 'ALT']

In [173]:
df = pd.merge(allele_df, vcf_df, how='inner', on=list(allele_df.columns))

In [174]:
df

,CHROM,POS,REF,ALT,ID,QUAL,FILTER,INFO,FORMAT,HG00096,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,22,16050075,A,G,rs587697622,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=8012;EA...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1,22,16050607,G,A,rs587720402,100,PASS,AC=5;AF=0.000998403;AN=5008;NS=2504;DP=20274;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,22,16051165,C,T,rs587731798,100,PASS,AC=2;AF=0.000399361;AN=5008;NS=2504;DP=23047;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [184]:
df.to_csv(None, header=None, index=None, sep='\t')

'22\t16050075\tA\tG\trs587697622\t100\tPASS\tAC=1;AF=0.000199681;AN=5008;NS=2504;DP=8012;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0;SAS_AF=0.001;AA=.|||;VT=SNP\tGT\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|0\t0|

In [181]:
# df.to_string(header=None, index=None, col_space=10).replace(' ' * 10, '\t')

'22   16050075\tA\tG  rs587697622        100       PASS  AC=1;AF=0.000199681;AN=5008;NS=2504;DP=8012;EA...         GT        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0        0|0   